In [1]:
import sqlite3
import numpy as np
import chess

# Function to convert FEN to input tensor
def fen_as_input(fen: str):
    board = chess.Board(fen)
    inputs = np.zeros(65, dtype=np.float16)
    for i in range(64):
        piece = board.piece_at(i)
        if piece is not None:
            inputs[i] = piece.piece_type * (1 if piece.color == chess.WHITE else -1)
    inputs[64] = 1 if board.turn == chess.WHITE else -1 # forgor this earlier ☠
    return inputs.tobytes()  # convert tensor to bytes

# Function to normalize eval field
def normalize_eval(eval):
    return max(min(eval / 10.0, 1.5), -1.5)

In [2]:
# Connect to existing DB using sqlite3
old_db = sqlite3.connect("../datasets/2021-07-31-lichess-evaluations-37MM.db/test.db")

# Connect to new DB using sqlite3
new_db = sqlite3.connect('new.db')

In [3]:
# Create table in new DB
new_db.execute("""
    CREATE TABLE Evaluations (
        id INTEGER PRIMARY KEY,
        fen TEXT NOT NULL,
        input_tensor BLOB NOT NULL,
        eval_for_white REAL NOT NULL
    )
""")

In [4]:
# Fetch data from old DB and insert into new DB
old_data = old_db.execute("SELECT id, fen, eval FROM Evaluations")
for id, fen, eval in old_data:
    new_db.execute("""
        INSERT INTO Evaluations (id, fen, input_tensor, eval_for_white)
        VALUES (?, ?, ?, ?)
    """, (id, fen, fen_as_input(fen), normalize_eval(eval)))

KeyboardInterrupt: 

In [5]:
# Commit changes and close connection
new_db.commit()
new_db.close()